# テーブル概要
accident:  
event:  
gv(general vehicle):車両一般  
ve(Exterior Vehicle):車両外部  
vi(Interior Vehicle):車両内部  
oa(OCCUPANT ASSESSMENT):乗員の調査  
oi(OCCUPANT INJURY):乗員の傷害(mergeに使用できるkeyの値が同一でも傷害箇所によってレコードが増加)  

# 最終的な作成データ
- Crash year 2010–2015
- Vehicle model year 2001–2015
- Light vehicles (passenger cars, pick-ups and mini-vans) 
- Non-ejected occupants
- Occupant age 15 or higher
- Occupants with known injury status or fatality

# ライブラリのインポート Pandasの表示設定
同一cellに複数テーブルを表示  
全カラムを表示  
最大表示行数:500  
1つのカラムの最大表示文字数:200  
floatの有効桁数:4  
色付き文字の出力:print(pycolor.RED + '文字列' + pycolor.END)  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import sys
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel
from IPython import embed
from IPython.core.interactiveshell import InteractiveShell
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 200)
pd.options.display.float_format = '{:.4g}'.format
class pycolor:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    PURPLE = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    END = '\033[0m'
    BOLD = '\038[1m'
    UNDERLINE = '\033[4m'
    INVISIBLE = '\033[08m'
    REVERCE = '\033[07m'

# NASS CDSデータの読み込み

In [ ]:
path = os.path.dirname(os.path.abspath('__file__'))
file_name = ['accident', 'event', 'gv', 'oa', 'oi', 've', 'vi']
cds_key = []
cds = {}
uyear = [str(x) for x in range(1, 16)]
for year in range(2001, 2016):
    for file in file_name:
        if year >= 2009:
            df = pd.read_sas(os.path.join(path, str(year), 'FormattedData', '{}.sas7bdat'.format(file)))
        elif year >= 2001:
            df = pd.read_sas(os.path.join(path, str(year), 'PCSAS', '{}.sas7bdat'.format(file)))
        cds_key.append('{}_{}'.format(file, year - 2000))
        cds['{}_{}'.format(file, year - 2000)] = df

## 整数の値が入っているはずなのに小数点以下の値が入っているもの，同じ値なのにpython内部で別の値として認識されているものを修正
## 複数のテーブルで重複しているが利用しないカラムの削除

In [ ]:
for col in ['PSU', 'VEHNO']:
    for y in uyear:
        for og in ['oa', 'gv', 've', 'vi']:
            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)
        if col !=  'VEHNO':
            og = 'accident'
            cds['{}_{}'.format(og, y)][col] = cds['{}_{}'.format(og, y)][col].astype(np.int64)
            
for y in uyear:
    y  = str(y)
    cds['accident_{}'.format(y)]['VEHFORMS'] = cds['accident_{}'.format(y)]['VEHFORMS'].astype(np.int64)

for col in ['CASENO', 'RATWGT', 'STRATIF', 'VERSION']:
    for y in range(10, 16):
        for og in ['oa', 'gv', 've', 'vi', 'accident', 'event', 'oi']:
            cds['{}_{}'.format(og, y)] = cds['{}_{}'.format(og, y)].drop(col, axis = 1)

## テーブル，年毎のレコード数を表示

In [ ]:
for year in uyear:
    print(year)
    for file in file_name:
        print('{}:'.format(file) + str(len(cds['{}_{}'.format(file, year)])))

## テーブル毎の結合keyを宣言

In [ ]:
merge_key = {}
for file in file_name:
    if file == 'accident' or file == 'event':
        merge_key[file] =  ['CASEID', 'PSU']
    if file == 'gv' or file == 've' or file == 'vi':
        merge_key[file] = ['CASEID', 'PSU', 'VEHNO']
    if file == 'oa' or file == 'oi':
        merge_key[file] = ['CASEID', 'PSU', 'VEHNO', 'OCCNO']

## テーブルの結合
oi,eventテーブルを除き,2010~2015年のデータをそれぞれ結合する

In [ ]:
cds_merge = {}
for year in uyear:
    cds_merge[year] = cds['oa_{}'.format(year)]
    for file in [x for x in file_name if not (x == 'oa' or x == 'oi' or x == 'event')]:
        if file != 'gv':
            cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'left')
        else:
            cds_merge[year] = pd.merge(cds_merge[year], cds['{}_{}'.format(file, year)], on = merge_key[file], how = 'inner')

## 編集用辞書データの作成

In [ ]:
cds_prepro = {}
for year in uyear:
    cds_prepro[year] = cds_merge[year]
    print(year,len(cds_prepro[year]))

## Car to Carの事故に限定
2台の事故のみに変更  

In [ ]:
for year in uyear:
    cds_prepro[year] = cds_prepro[year].query('VEHFORMS == 2')
    print(year, len(cds_prepro[year]))

## もう一方の車両の速度を示すカラムを作成

In [ ]:
#合ってるか怪しい
for year in uyear:
    sp = cds_prepro[year][['CASEID', 'PSU', 'VEHNO', 'DVTOTAL']]
    sp = sp.drop_duplicates(subset = ['CASEID', 'PSU', 'VEHNO'])
    sp1 = sp.query('VEHNO == 1')
    sp1 = sp1.drop('VEHNO', axis  = 1)
    sp1 = sp1.rename(columns = {'DVTOTAL': 'SP1'})
    sp2 = sp.query('VEHNO == 2')
    sp2 = sp2.drop('VEHNO', axis  = 1)
    sp2 = sp2.rename(columns = {'DVTOTAL': 'SP2'})
    sp = pd.merge(sp1, sp2, on = ['CASEID', 'PSU'], how = 'inner')
    v1o = sp[['CASEID', 'PSU', 'SP2']]
    v1o['VEHNO'] = 1
    v2o = sp[['CASEID', 'PSU', 'SP1']]
    v2o['VEHNO'] = 2   
    cds_prepro[year] = pd.merge(cds_prepro[year], v1o, on = ['CASEID', 'PSU', 'VEHNO'], how = 'left')
    cds_prepro[year] = pd.merge(cds_prepro[year], v2o, on = ['CASEID', 'PSU', 'VEHNO'], how = 'left')
    cds_prepro[year] = cds_prepro[year].fillna({'SP1': 0,  'SP2':  0})
    cds_prepro[year]['otbsp'] = cds_prepro[year]['SP1'] + cds_prepro[year]['SP2'] 
    cds_prepro[year] = cds_prepro[year].drop(['SP1', 'SP2'], axis = 1)

## エアバッグ利用可能・シートベルト着用・15歳以上に限定
ベルトは3点ベルトのみ着用しているとみなす

In [ ]:
for year in uyear:
    cds_prepro[year] = cds_prepro[year].query('MAIS <= 6 & PARUSE == 4 & AGE >= 15')
    print(year, len(cds_prepro[year]))

## 利用可能な特徴量を全て抽出

In [ ]:
pcol = 'MAIS'
use_col = [pcol, \
           'CASEID', 'PSU', 'VEHNO', \
           'YEAR', 'MONTH', 'TIME', 'DAYWEEK', 'DVTOTAL', \
           'MODEL', 'MODELYR', 'BODYTYPE', 'ALIGNMNT', 'ANTILOCK', 'CARGOWGT', 'CONDTREE', 'CURBWGT', 'FOURWHDR', 'FRTWHLDR', 'FUELCODE', 'LGTCOND', 'PROFILE', 'RELINTER', 'RESTYPE', 'SPLIMIT', 'SURCOND', 'SURTYPE', 'TRAFCONT', 'TRAFFLOW', 'TRAVELSP', 'TRCTLFCT', 'VEHTYPE', 'VEHUSE', 'VEHWGT', 'WGTCDTR', 'WHLDRWHL', 'otbdytyp', 'otvehwgt', \
           'FUELCAP1', 'FUELCAP2', 'FUELLOC1', 'FUELLOC2', 'FUELTYP1', 'FUELTYP2','FUELTNK1', 'FUELTNK2', 'ORIGAVTW', 'PDOF1', 'SHL1', 'WHEELBAS', \
           'GLTYPWS', 'GLTYPLF', 'GLTYPLR', 'GLTYPRF', 'GLTYPRR', 'GLTYPBL', 'GLTYPRUF', 'GLTYPOTH', 'GLPREWS', 'GLPRELF', 'GLPRELR', 'GLPRERF', 'GLPRERR', 'GLPREBL', 'GLPRERUF', 'GLPREOTH', 'COLUMTYP', 'COLMTELE', 'COLMTILT', 'ODOMETER', 'ADAPTEQ', \
           'AGE', 'BAGAVAIL', 'BAGAVOTH', 'BAGMAINT', 'BAGTYPE', 'BELTANCH', 'HEIGHT', 'MANAVAIL', 'MANUSE', 'POSTURE',  'ROLE', 'SEATPOS', 'SEATRACK', 'SEATTYPE', 'SEX', 'STORIENT', 'WEIGHT', \
           'otbsp']
for year in uyear:
    cds_prepro[year] = cds_prepro[year][use_col]

## 1~15年の前データ確認用データフレームの作成

In [ ]:
columns  = use_col.remove('otbsp')
cds_all  = cds_merge['1'][use_col]
for year in range(2, 16):
    year = str(year)
    cds_all = pd.concat([cds_all, cds_merge[year][use_col]])

In [ ]:
cds_all.head(50)

## SURCONDの年毎に違う値を修正
2009年以降がより細分化されて記録されているので、2008年の基準に統一

In [ ]:
for year in range(9, 16):
    year = str(year)
    cds_prepro[year].loc[cds_prepro[year]['SURCOND'] == 4, 'SURCOND'] =  3
    cds_prepro[year].loc[cds_prepro[year]['SURCOND'] == 5, 'SURCOND'] = 4
    cds_prepro[year].loc[(cds_prepro[year]['SURCOND'] >= 7)  & (cds_prepro[year]['SURCOND'] <= 9), 'SURCOND'] = 5
    cds_prepro[year].loc[(cds_prepro[year]['SURCOND'] > 87) | (cds_prepro[year]['SURCOND']  == 6), 'SURCOND'] = 8

## MAIS → MAIS3+, TIME → 時間帯,  SEX → 性別・妊娠の有無

In [ ]:
for year in uyear:
    cds_prepro[year].loc[cds_prepro[year][pcol] < 3, 'MAIS3+'] = 0
    cds_prepro[year].loc[cds_prepro[year][pcol] >= 3, 'MAIS3+'] = 1
    cds_prepro[year].loc[(cds_prepro[year]['TIME'] >= 600) & (cds_prepro[year]['TIME'] < 900), 'TZONE'] = 1
    cds_prepro[year].loc[(cds_prepro[year]['TIME'] >= 900) & (cds_prepro[year]['TIME'] < 1200), 'TZONE'] = 2
    cds_prepro[year].loc[(cds_prepro[year]['TIME'] >= 1200) & (cds_prepro[year]['TIME'] < 1500), 'TZONE'] = 3
    cds_prepro[year].loc[(cds_prepro[year]['TIME'] >= 1500) & (cds_prepro[year]['TIME'] < 1800), 'TZONE'] = 4
    cds_prepro[year].loc[(cds_prepro[year]['TIME'] >= 1800) & (cds_prepro[year]['TIME'] < 2100), 'TZONE'] = 5
    cds_prepro[year].loc[cds_prepro[year]['TIME'] >= 2100, 'TZONE'] = 6
    cds_prepro[year].loc[cds_prepro[year]['TIME'] < 300, 'TZONE'] = 7
    cds_prepro[year].loc[(cds_prepro[year]['TIME'] >= 300) & (cds_prepro[year]['TIME'] < 600), 'TZONE'] = 8
    cds_prepro[year] = cds_prepro[year].drop(['TIME'], axis = 1)
    cds_prepro[year].loc[cds_prepro[year]['SEX'] > 2, 'PREG'] = 1
    cds_prepro[year].loc[cds_prepro[year]['SEX'] >= 2, 'SEX'] = 0
    cds_prepro[year] = cds_prepro[year].fillna({'PREG' : 0})

## 車両毎にMAISの最大値のみ残す
MAISでソートし, VEHNOが重複しているレコードを上にあるものを残して削除  
MAIS3+のカラムを作成したのでMAISのカラムはドロップ

In [ ]:
for year in uyear:
    print(pycolor.RED + year + pycolor.END)
    print(len(cds_prepro[year]))
    cds_prepro[year] = cds_prepro[year].sort_values(by = pcol, ascending = False)
    cds_prepro[year] = cds_prepro[year].drop_duplicates(subset = ['CASEID', 'PSU', 'VEHNO'])
    cds_prepro[year] = cds_prepro[year].drop([pcol, 'PSU', 'CASEID', 'VEHNO'], axis = 1)
    print(len(cds_prepro[year]))

## 1~15年を結合

In [ ]:
X_train = cds_prepro['1']
for year in range(2, 15):
    year = str(year)
    X_train = pd.concat([X_train, cds_prepro[year]])
X_test = cds_prepro['15']
len(X_train)

In [ ]:
X_train.head()

## 整数値に変換し直す

In [ ]:
X_test['ADAPTEQ'] = X_test['ADAPTEQ'].astype(np.int64)
X_test['COLUMTYP'] = X_test['COLUMTYP'].astype(np.int64)
X_test['SEATRACK'] = X_test['SEATRACK'].astype(np.int64)
X_train['ADAPTEQ'] = X_train['ADAPTEQ'].astype(np.int64)
X_train['COLUMTYP'] = X_train['COLUMTYP'].astype(np.int64)
X_train['SEATRACK'] = X_train['SEATRACK'].astype(np.int64)

In [ ]:
dtype = X_train.dtypes
for i in X_train.columns:
    if dtype[i] == 'object':
        label, unique = pd.factorize(X_train[i])
        X_train[i] = label
        X_train.loc[X_train[i] == -1, i] = pd.np.nan
        label, unique = pd.factorize(X_test[i])
        X_test[i] = label
        X_test.loc[X_test[i] == -1, i] = pd.np.nan

## NaNを中央値で置換

In [ ]:
cds_md = X_train.median()
X_train = X_train.fillna(cds_md)
X_test = X_test.fillna(cds_md)
cds_lasso = pd.concat([X_train, X_test])

In [ ]:
X_test.head()

## 説明変数と目的変数に分割

In [ ]:
y_train = X_train['MAIS3+']
X_train = X_train.drop(['MAIS3+'], axis = 1)
y_test = X_test['MAIS3+']
X_test = X_test.drop(['MAIS3+'], axis = 1)

## ランダムフォレストで推定

In [ ]:
clabel = {0:1, 1:500}
grid_param = {'n_estimators': [500], 'max_depth': [10, 20, 30]}
forest = GridSearchCV(RandomForestClassifier(random_state = 0, class_weight = clabel), grid_param, cv = 5, scoring = 'roc_auc')
forest.fit(X_train, y_train)
predict = forest.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, predict, pos_label = 1)
print('best parameter:', forest.best_params_)
print('auc:', auc(fpr, tpr))
print('accuracy', accuracy_score(predict, y_test))
fti = forest.best_estimator_.feature_importances_
columns = list(X_train.columns.values)
im_df = pd.DataFrame({'Feature':columns,
                     'importance':fti})
im_df = im_df.sort_values('importance', ascending = False)
im_df.head(50)

In [ ]:
list(predict).count(0)
list(predict).count(1)

##  ダミー変数化 

In [ ]:
dummy_col = ['DAYWEEK', 'MONTH',  \
             'MODEL', 'BODYTYPE', 'ALIGNMNT', 'ANTILOCK', 'CONDTREE', 'FOURWHDR', 'FRTWHLDR', 'FUELCODE', 'LGTCOND', 'PROFILE', 'RELINTER', 'RESTYPE', 'SURCOND', 'SURTYPE', 'TRAFCONT', 'TRAFFLOW', 'TRCTLFCT', 'VEHTYPE', 'VEHUSE', 'otbdytyp',\
             'FUELCAP1', 'FUELCAP2', 'FUELLOC1', 'FUELLOC2', 'FUELTYP1', 'FUELTYP2', 'FUELTNK1', 'FUELTNK2', 'PDOF1', 'SHL1', \
             'GLTYPWS', 'GLTYPLF', 'GLTYPLR', 'GLTYPRF', 'GLTYPRR', 'GLTYPBL', 'GLTYPRUF', 'GLTYPOTH', 'GLPREWS', 'GLPRELF', 'GLPRELR', 'GLPRERF', 'GLPRERR', 'GLPREBL', 'GLPRERUF', 'GLPREOTH', 'COLUMTYP', 'COLMTELE', 'COLMTILT', 'ADAPTEQ', \
             'BAGAVAIL', 'BAGAVOTH', 'BAGMAINT', 'BAGTYPE', 'BELTANCH', 'MANAVAIL', 'MANUSE',  'POSTURE',  'ROLE', 'SEATPOS', 'SEATRACK', 'SEATTYPE', 'STORIENT']
cds_dummy = pd.get_dummies(cds_lasso, drop_first = False, columns  = dummy_col)

## テストデータと訓練データに分割

In [ ]:
cds_test = cds_dummy.query('YEAR == 2015')
cds_train = cds_dummy.query('YEAR < 2015')

In [ ]:
X_train = cds_train.drop(['MAIS3+'], axis = 1)
y_train = cds_train['MAIS3+']
X_test = cds_test.drop(['MAIS3+'], axis = 1)
y_test = cds_test['MAIS3+']

In [ ]:
list(predict).count(0)
list(predict).count(1)

## 標準化
量的データを平均0分散1に標準化  

In [ ]:
st_col = ['YEAR', \
           'MODELYR', 'CARGOWGT','CURBWGT', 'SPLIMIT', 'TRAVELSP', 'VEHWGT', 'WGTCDTR', 'WHLDRWHL','otvehwgt', 'DVTOTAL', \
           'ORIGAVTW', 'WHEELBAS', \
           'ODOMETER', \
           'AGE', 'HEIGHT', 'WEIGHT', \
           'otbsp']
scaler = StandardScaler(copy = True, with_mean = True)
scaler.fit(X_train[st_col])
X_train_st = pd.DataFrame(scaler.transform(X_train[st_col]), columns = st_col)
X_test_st = pd.DataFrame(scaler.transform(X_test[st_col]), columns = st_col)
X_train = X_train.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)
X_train[st_col] = X_train_st
X_test[st_col] = X_test_st

In [ ]:
y_train.value_counts()
y_test.value_counts()

## Lassoで特徴量選択

In [ ]:
lasso_cv = linear_model.LassoCV(alphas = 10 ** np.arange(-6, 1, 0.1), cv = 5)
lasso_cv.fit(X_train, y_train)
print(lasso_cv.alpha_)

In [ ]:
for year in uyear:
    X_train['ADAPTEQ'] = X_train['ADAPTEQ'].astype(np.int64)
    X_train['COLUMTYP'] = X_train['COLUMTYP'].astype(np.int64)
    X_train['SEATRACK'] = X_train['SEATRACK'].astype(np.int64)

In [ ]:
lasso_cv.score(X_test, y_test)
lasso_cv.score(X_train, y_train)

In [ ]:
f_df = pd.DataFrame({'Feature' : list(X_train.columns.values),
                     'coef' : lasso_cv.coef_})
f_df = f_df.sort_values('coef', ascending = False)

In [ ]:
f_df.head(50)

In [ ]:
len(f_df[f_df.coef != 0])
len(f_df)
selected_f = list(f_df[f_df.coef != 0]['Feature'])

In [ ]:
X_train_s = X_train[selected_f]
X_test_s = X_test[selected_f]

## 選択した特徴量で再度ランダムフォレストで推定 

In [ ]:
forest = RandomForestClassifier(n_estimators = 100, random_state = 0, max_depth = 100)
forest.fit(X_train_s, y_train)
predict = forest.predict(X_test_s)
fpr, tpr, thresholds = roc_curve(y_test, predict, pos_label = 1)
print('auc:', auc(fpr, tpr))
print('accuracy', accuracy_score(predict, y_test))
fti = forest.feature_importances_
columns = list(X_train_s.columns.values)
im_df = pd.DataFrame({'Feature':columns,
                     'importance':fti})
im_df = im_df.sort_values('importance', ascending = False)
im_df.head(50)

In [ ]:
X_train.head)

## L1正則化ロジスティック回帰で推定

In [ ]:
grid_param = {'C': [0.06], 'solver': ['liblinear']}
grid_search = GridSearchCV(LogisticRegression(random_state = 0, penalty = 'l1'), grid_param, cv = 5)
grid_search.fit(X_train, y_train)
print('best parameter:', grid_search.best_params_)
predict = grid_search.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, predict, pos_label = 1)
print('auc:', auc(fpr, tpr))
print('accuracy:', accuracy_score(predict, y_test))

In [ ]:
f_df = pd.DataFrame({'Feature' : list(X_train.columns.values),
                     'coef' : grid_search.best_estimator_.coef_[0]})
f_df = f_df.sort_values('coef', ascending = False)
F_df = f_df.sort_values('coef', ascending = True)
f_df.head(50)
F_df.head(70)
log_s_f = list(f_df[f_df.coef != 0]['Feature'])
len(f_df)
len(log_s_f)

## 係数を絶対値で表記しソート

In [ ]:
df_abs = lambda x: -x if x < 0 else x
f_abs = f_df
f_abs['coef'] = f_abs['coef'].map(df_abs)
a_abs = f_abs.sort_values('coef', ascending = False)
f_abs.head(40)

## 選択した特徴量で再度ランダムフォレストで推定

In [ ]:
grid_param = {'n_estimators': [400], 'max_depth': [100]}
grid_search = GridSearchCV(RandomForestClassifier(random_state = 0), grid_param, cv = 5)
grid_search.fit(X_train_s, y_train)
predict = grid_search.predict(X_test_s)
fpr, tpr, thresholds = roc_curve(y_test, predict, pos_label = 1)
print('best parameter:', grid_search.best_params_)
print('auc:', auc(fpr, tpr))
print('accuracy', accuracy_score(predict, y_test))
fti = grid_search.best_estimator_.feature_importances_
columns = list(X_train_s.columns.values)
im_df = pd.DataFrame({'Feature':columns,
                     'importance':fti})
im_df = im_df.sort_values('importance', ascending = False)
im_df.head(50)

In [ ]:
grid_param = {'alphas' : [0.001, 0.01, 0.1, 1, 10]}
lasso_cv = GridSearchCV(Lasso(random_state = 0, clas))